In [171]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# from torcheval.metrics.functional import binary_accuracy


In [172]:
# import data.tsv file and create a dataframe

df = pd.read_csv('data.tsv', sep='\t')

# open file data_info.txt and read the first line

sequence_length = None
with open('data_info.txt', 'r') as f:
    line = f.readline()
    split_line_on_collen = line.split(':')
    sequence_length = int(split_line_on_collen[1])
    


train_df = df.sample(frac=0.8, random_state=0)
test_df = df.drop(train_df.index)


# drop the collumn Y from the dataframe

train_x = train_df.drop(['Y'], axis=1).iloc[1:]
train_y = train_df['Y'].iloc[1:]

# make the tran_x data rows have the same length as the sequence_length by padding with [0,0,0] for each collumn

# train_x = train_x.reindex(range(sequence_length), fill_value=[0,0,0])

test_x = test_df.drop(['Y'], axis=1)


test_y = test_df['Y']



In [173]:
def convert_rows_to_nupy_array(df):
    numpy_array = []
    for index, row in df.iterrows():
        row_array = []
        for col in row.iteritems():
            col_float = []
            for item in col[1].split(','):
                col_float.append(float(item.replace('[', '').replace(']', '')))
            row_array.append(col_float)
        numpy_array.append(row_array)
    return np.array(numpy_array)

train_x_numpy = convert_rows_to_nupy_array(train_x)
test_x_numpy = convert_rows_to_nupy_array(test_x)

print(train_x_numpy.shape)
print(test_x_numpy[1][5])
            

(2057, 749, 3)
[ 0.51171875 -0.10406494 -0.10406494]


##### Remove Padding from numpy

In [174]:
train_y_no_pad = train_y
test_y_no_pad = test_y

# remove the numpy array where you hava a [0,0,0] array

test_np_array = np.array([[1,2,1], [0,0,0], [1,2,1]])

# remove the [0,0,0] array from the numpy array test_np_array

test_np_array = test_np_array[test_np_array[:,0] != 0]



train_x_numpy_no_pad = []
# row_index = 0
for row in range(len(train_x_numpy)):
    temp = train_x_numpy[row][train_x_numpy[row][:,0] != 0]
    temp_list = []
    for col in temp:
        item_list = []
        for item in col:
            item_list.append(item)
        temp_list.append(item_list)
    train_x_numpy_no_pad.append(temp_list)
train_x_numpy_no_pad = np.array(train_x_numpy_no_pad)
print(train_x_numpy_no_pad.shape)


(2057,)


/tmp/ipykernel_13502/3299080516.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_x_numpy_no_pad = np.array(train_x_numpy_no_pad)


## Train

In [175]:
# Dummy Classifer

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

dummy_clf = DummyClassifier(strategy="most_frequent")
print(train_x_numpy.shape, train_y.shape)
dummy_clf.fit(train_x_numpy, train_y)
y_pred = dummy_clf.predict(test_x_numpy)
print(accuracy_score(test_y, y_pred))

(2057, 749, 3) (2057,)
0.7704280155642024


In [176]:

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, input,target, seq_len):
        self.input = input
        self.target = target
        self.seq_len = seq_len
    def __getitem__(self, item):
        # print("input item", self.input[item])
        return self.input[item], self.target.iloc[item]

        # return input[item:item+self.seq_len], input[item+self.seq_len]
    def __len__(self):
        return  self.input.shape[0]

# test_input = np.arange(1,8).reshape(-1,1)
# input = torch.tensor(test_input, dtype=torch.float)

# dataset = MyDataset(input, 3)

# dl = torch.utils.data.DataLoader(dataset, batch_size=1)
# for inp, label in dl:
#     print("inp shape",inp.shape)
#     print("inp",inp.numpy())


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


input_size = 3
num_classes = 1
hidden_size = 256
num_layers = 2
learning_rate = 0.005

class SitUpDetector(nn.Module):
    def __init__(self,input_size, num_classes, hidden_size, num_layers,drop_prob=0.5):
        super(SitUpDetector, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, _ = self.lstm(x, hidden)
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.dropout(out)
        out = self.fc(out)
        out = self.sigmoid(out)

        out = out.view(batch_size, -1)
        out = out[:, -1]
        return out,hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.num_layers, batch_size, self.hidden_size).zero_().float().to(device),
                      weight.new(self.num_layers, batch_size, self.hidden_size).zero_().float().to(device)) 
        return hidden
    


model = SitUpDetector(input_size, num_classes, hidden_size, num_layers).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# reduce size of train_x_numpy_no_pad to 1000
train_x_numpy = train_x_numpy[:50]
train_y = train_y[:50]

test_x_numpy = test_x_numpy[:50]
test_y = test_y[:50]


# print how many 1 and 0 in the train_y
print(train_y.value_counts())

batch_size = 1

myDataset = MyDataset(train_x_numpy,train_y, sequence_length)
train_loader = torch.utils.data.DataLoader(dataset=myDataset, batch_size=batch_size)

losses = []
loss = None
num_epochs = 10
clip = 5
valid_loss_min = np.Inf
total_steps = len(train_loader)

model.train()
for epoch in range(num_epochs):
    h = model.init_hidden(batch_size)
    for i, (features, target) in enumerate(train_loader):


        h = tuple([e.data for e in h])

        # convert Double tensor to Float tensor
        # features = features.float()

        features, target = features.to(device), target.to(device)
        model.zero_grad()
        outputs, h = model(features.float(), h)
        loss = criterion(outputs, target.float())
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        loss.backward()

        optimizer.step()
        if (i+1) % 50 == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inp, lab in train_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, num_epochs),
                  "Step: {}...".format(i),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

# graph the losses 

# get the lenght of the losses array

# create an array of the same length as the losses array with index values
x = np.arange(0,num_epochs)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(x,losses)
plt.show()






# Confusion Matrix
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt

# cm = confusion_matrix(test_y, all_predictions)
# print(cm)
# sns.heatmap(cm, annot=True, fmt="d")

torch.save(model.state_dict(), 'sit_up_detector.pth')






0    41
1     9
Name: Y, dtype: int64


RuntimeError: expected scalar type Double but found Float

In [ ]:
# Check accuracy
test_dataset = MyDataset(test_x_numpy, test_y, sequence_length)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1)
n_correct = 0
n_samples = 0
model.eval()

all_predictions = []

test_losses = []
num_correct = 0
h = model.init_hidden(batch_size)

model.eval()
for inputs, labels in test_dataloader:
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    output, h = model(inputs, h)
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    pred = torch.round(output.squeeze())  # Rounds the output to 0/1
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_dataloader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))

torch.Size([749, 3])


RuntimeError: input must have 3 dimensions, got 2

# Train with no Padding

In [ ]:
class MyDatasetVariableSequenceLength(torch.utils.data.Dataset):
    def __init__(self, input,target, seq_len):
        self.input = input
        self.target = target
        self.seq_len = seq_len
    def __getitem__(self, item):
        # print("input item", self.input[item])
        return self.input[item], self.target.iloc[item]

        # return input[item:item+self.seq_len], input[item+self.seq_len]
    def __len__(self):
        return self.input.shape[0]

In [ ]:
#  create pytorch lstm variable recurrent classifier

input_size = 3
num_classes = 1
hidden_size = 256
num_layers = 2
learning_rate = 0.001


class SitUpDetectorVariableSequenceLength(nn.Module):
    def __init__(self,input_size, num_classes, hidden_size, num_layers):
        super(SitUpDetectorVariableSequenceLength, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        outputs = []
        
        # x = torch.squeeze(x) # remove the batch dimension maybe sometime allow for batch size > 1
        
        for xx in x:
            s, n = xx.shape
            print(xx.shape)
            out, _ = self.lstm(xx)
            out = out.view(s, -1)
            outputs.append(out)

        outputs = torch.stack(outputs)

        out = self.fc(outputs)
        return out

# train SitUpDetectorVariableInput
model = SitUpDetectorVariableSequenceLength(input_size, num_classes, hidden_size, num_layers).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

myDatasetVariableSequenceLengthDataset = MyDatasetVariableSequenceLength(train_x_numpy,train_y, sequence_length)
train_loader = torch.utils.data.DataLoader(dataset=myDatasetVariableSequenceLengthDataset, batch_size=1, shuffle=False)

losses = []
loss = None
num_epochs = 1
total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (features, target) in enumerate(train_loader):

        # convert Double tensor to Float tensor
        features = features.float()

        outputs = model(features)
        
        loss = criterion(outputs, target.float())

        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_steps, loss.item()))
    losses.append(loss.item())



torch.Size([749, 3])


RuntimeError: input must have 3 dimensions, got 2

# univariate classifier

In [ ]:
# univariate time series classifier



